# COVID-19 Python Model

In [4]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
# This makes the plots appear inside the notebook
%matplotlib inline
from scipy.integrate import odeint

## Parameters

Population parameters

In [12]:
agepars_meanage_in=np.arange(5,95,10)
agepars_highage=np.arange(9,99,10)
agepars_lowage=np.arange(0,90,10)

#Data from 2018 census
population_N= 10666108

population_agefrac_in = [0.126,0.137,0.139,0.132,0.130,0.129,0.104,0.061,0.036,0.007]
myInt = sum(population_agefrac_in)
population_agefrac = [x / myInt for x in population_agefrac_in]

agepars_meanage= [a * b for a, b in zip(agepars_meanage_in, population_agefrac)]
population_meanage = sum(meanage)

Check population data - this must sum to ~1.00

In [13]:
x = (sum(population_agefrac))
y = format(x,'.5f')
yy = format(1,'.5f')
bool(y==yy)

True

Basic parameters

In [14]:
pars_gamma_e=1/4;   #Transition to infectiousness
pars_gamma_a=1/6;   #Resolution rate for asymptomatic
pars_gamma_s=1/6;   #Resolution rate for symptomatic
pars_gamma_h=1/10;  #Resolution rate in hospitals
pars_beta_a=4/10;   #Transmission for asymptomatic
pars_beta_s=8/10;   #Transmission for symptomatic

pars_p=[0.95,0.95,0.90,0.8,0.7,0.6,0.4,0.2,0.2,0.2]         #Fraction asymptomatic

pars_overall_p= sum([a * b for a, b in zip(pars_p, population_agefrac)])

pars_Itrigger = 500000/population_N #Trigger at 5000 total cases, irrespective of type

Age stratification

In [15]:
agepars_hosp_frac_in=[0.1,0.3,1.2,3.2,4.9,10.2,16.6,24.3,27.3,27.3]
agepars_hosp_frac = [x / 100 for x in agepars_hosp_frac_in]

agepars_hosp_crit_in=[5,5,5,5,6.3,12.2,27.4,43.2,70.9,70.9]
agepars_hosp_crit = [x / 100 for x in agepars_hosp_crit_in]

agepars_crit_die= 0.5*np.ones(len(meanage)+1) ## CHECK
agepars_num_ages = len(meanage);

N=agepars_num_ages;
agepars_S_ids= (1,N)
agepars_E_ids= ((N+1),(2*N))
agepars_Ia_ids=((2*N+1),(3*N))
agepars_Is_ids=((3*N+1),(4*N))
agepars_Ihsub_ids=((4*N+1),(5*N))
agepars_Ihcri_ids=((5*N+1),(6*N))
agepars_R_ids=((6*N+1),(7*N))
agepars_D_ids=((7*N+1),(8*N))
agepars_Hcum_ids=((8*N+1),(9*N))

agepars_IFR_2= [a * b * c * d for a, b, c, d in zip(population_agefrac, agepars_hosp_frac, agepars_hosp_crit, agepars_crit_die)]
pp = [a-b for a, b in zip(np.ones(len(pars_p)), pars_p)]
agepars_IFR_1= [a*b for a,b in zip(agepars_IFR_2,pp)]
agepars_IFR = sum(agepars_IFR_1)

Epidemiological parameters

In [16]:
pars_Ra=pars_beta_a/pars_gamma_a;
pars_Rs=pars_beta_s/pars_gamma_s;


x = [a-b for a, b in zip(np.ones(len(pars_p)), pars_p)] #1-pars_p
y = [a*b for a,b in zip(x,population_agefrac)] #(1-pars_p*pop_agefrac)
z = [a*pars_Rs for a in y] #(1-pars_p*pop_agefrac*pars_Rs)
m = [a*b*pars_Ra for a,b in zip(pars_p,population_agefrac)] #(pars_p*pop_agefrac*pars_Ra)
pars_R0 = [a*b for a,b in zip(z,m)]

#pars_bau=pars;

## Initial Conditions

TO COMPELTE -- Population initial conditions

In [17]:
#Init the population - baseline
#pen plus hospitals
#SEIaIS (open) and then I_ha I_hs and then R (open) and D (cumulative) age stratified
tmpzeros = np.zeros(len(agepars_meanage),dtype = int)
outbreak_y0=np.concatenate([population_agefrac,tmpzeros,tmpzeros,tmpzeros,tmpzeros,tmpzeros,tmpzeros,tmpzeros,tmpzeros])

#Initiate an outbreak with 500 symptomatic current cases and 7500 asymptomatic cases
#effective 8000 total and 25 deaths (based on GA estimates)
#Initiate an outbreak
outbreak_y0=[a*population_N for a in outbreak_y0]
outbreak_y0[2] = outbreak_y0[2]-1

#outbreak_y0[12]=1 #CHECK THESE!
outbreak_y0=[a/population_N for a in outbreak_y0]

outbreak_pTime=365;
outbreak_pNear=30;
outbreak_pshift=0;

## Mathematical Model

State level model function (S,E,Ia,Is,R)

In [18]:
def dY_dt(Y, t):
    ## Algebraic equations ##
    Ia=sum(agepars_Ia_ids)
    Is=sum(agepars_Is_ids)
    R = sum(agepars_R_ids)
    S = sum(agepars_S_ids)
    E = sum(agepars_E_ids)
    
    ## Trigger Loop ##
    Itot = 1-sum(Y[0])
    value = Itot - pars_Itrigger
    
    if value < 0:
        trigger = 0
    else:
        trigger = 1
    
    ## Differential variables ##
    return [
            #Y[0] = S population = agepars_S_ids
            -pars_beta_a*Y[0]*Ia - pars_beta_s*Y[0]*Is,
        
            #Y[1] = E population
            pars_beta_a*Y[0]*Ia + pars_beta_s*Y[0]*Is - pars_gamma_e*Y[1],
        
            #Y[2] = Ia population ## CHECK! -maybe need some element-wise multiplication
            np.transpose(pars_p)*pars_gamma_e*Y[1]-pars_gamma_a*Y[2],
        
            #Y[3] = Is population ## CHECK! -maybe need some element-wise multiplication
            np.transpose(np.ones(len(pars_p))-pars_p)*pars_gamma_e*Y[3] - pars_gamma_s*Y[3],
        
            #Y[4] = Ihsub population
            np.transpose(agepars_hosp_frac)*(1-np.transpose(agepars_hosp_crit))*pars_gamma_s*Y[3] - pars_gamma_h*Y[4],
        
            #Y[5] = Ihcri population
            np.transpose(agepars_hosp_frac)*(1-np.transpose(agepars_hosp_crit))*pars_gamma_s*Y[3] - pars_gamma_h*Y[5],
        
            #Y[6] = R population
            (pars_gamma_a*Y[2]) + (pars_gamma_s*Y[3]*(1-np.transpose(agepars_hosp_frac))) + (pars_gamma_h*Y[4]) + (pars_gamma_h*Y[5])*(1-np.transpose(agepars_crit_die)),
        
            #Y[7] = D population
            pars_gamma_h*Y[5]*np.transpose(agepars_crit_die),
        
            #Y[8] = Hcum population
            (np.transpose(agepars_hosp_frac)*(1-np.transpose(agepars_hosp_crit))*pars_gamma_s*Y[3]) + (np.transpose(agepars_hosp_frac)*np.transpose(agepars_hosp_crit)*pars_gamma_s*Y[3]),
            
            trigger
           ]


Simulating ODE model - COMPLETE - ODE INITIAL CONDS and SOLVER SETTINGS

In [3]:
ts = np.linspace(0, outbreak_pTime, 0.1)
Ys = odeint(dY_dt, outbreak_y0, ts) #solver settings not as defined as per matlab

#Simulation statistics
stats_Dcum = sum(Ys[:,7])
stats_Hcum = sum(Ys[:,8])

#% Sims - Get to Crossing
#opts=odeset('reltol',1e-8,'maxstep',0.1,'events',@intervene_trigger);
#[tpre,ypre,te,ye,ie]=ode45(@covid_model_ga,[0:1:outbreak.pTime], outbreak.y0,opts,pars,agepars);


NameError: name 'np' is not defined

In [2]:
Ys[1]

NameError: name 'Ys' is not defined

In [ ]:
fig = plt.figure()
ax1 = fig.add_axes([0.1, 0.5, 0.8, 0.4],
                   xticklabels=[], ylim=(0, 10))
ax2 = fig.add_axes([0.1, 0.1, 0.8, 0.4],
                   ylim=(0, 10))


Plotting

In [ ]:
fig = plt.figure()
ax1 = fig.add_axes([0.1, 0.5, 0.8, 0.4],
                   xticklabels=[], ylim=(0, 10))
ax2 = fig.add_axes([0.1, 0.1, 0.8, 0.4],
                   ylim=(0, 10))

x = np.linspace(0, 10)
ax1.plot(Ps[:,6])
ax2.plot(Ps[:,7]);
plt.xlabel("Time")
plt.ylabel("Absolute Cell Number")
plt.legend();